In [19]:
import tensorflow as tf
import numpy as np
from utils.utils import voxel_grid
import os
from sklearn.preprocessing import LabelEncoder, OneHotEncoder



In [27]:
#Helper functions (layer creation)

"""
    Default stride size is filterSize, which means no overlapping.
"""
def conv3d(x, numFilters, filterSize, stride=False, padding='VALID'):
    if(not stride):
        stride = filterSize
    inChannels = x.get_shape()[-1].value
    #The filter weights must be of the form (width, height, depth, channels, outputChannels[numberOfFilters])
    filter=tf.Variable(tf.truncated_normal(shape=[3,3,3,inChannels,numFilters], dtype=tf.float32))
    return tf.nn.conv3d(input=x, filter=filter, \
                        strides=[1,stride,stride,stride,1], padding=padding)


def flatten(x):
    return tf.reshape(x, [-1,1])

def fully_connected(x, size, activation=tf.nn.relu):
    b = tf.Variable(tf.zeros([size]))
    weights = tf.Variable(tf.truncated_normal([x.get_shape()[1].value, size]))
    return activation(b + tf.matmul(x, weights))


#Helper functions batch reading

object_csvs = list(filter(lambda x: ".csv" in x, os.listdir("../dataset/objects")))
def get_batch(size=12, ind=0):
    
    mini_batch_x = []
    for i in range(size):
        mini_batch_x.append(np.genfromtxt("../dataset/objects/"+object_csvs[ind+i], delimiter=",")[:,3:6])
        #mini_batch_x[i].append(object_csvs[ind+i].split(".")[0])
        #write code for labels
    
    
    return np.array(mini_batch_x)


get_batch(12,0)



array([ array([[ 6.21931,  2.71008,  1.14817],
       [ 6.43505,  2.59084,  1.50079],
       [ 5.81078,  2.59336,  2.491  ],
       ..., 
       [ 2.03256,  4.02662,  1.80329],
       [ 2.02302,  4.07075,  1.81764],
       [ 2.02285,  4.13483,  1.84105]]),
       array([[ 5.99372,  2.65313,  2.56703],
       [ 5.8246 ,  2.62159,  2.50059],
       [ 6.3875 ,  2.856  ,  1.18525],
       ..., 
       [ 2.25128,  3.89571,  1.79875],
       [ 2.25817,  3.96435,  1.82443],
       [ 2.20581,  3.93013,  1.80177]]),
       array([[ 2.25074, -6.8757 ,  2.41538],
       [ 2.28603, -6.83809,  2.40704],
       [ 2.27483, -6.31062,  2.23699],
       ..., 
       [ 5.16527, -5.61805,  2.63141],
       [ 5.22484, -5.6118 ,  2.64395],
       [ 5.27937, -5.59952,  2.65384]]),
       array([[-18.8331  ,   9.8232  ,   1.72129 ],
       [-19.051   ,   9.82274 ,   1.73822 ],
       [-18.8244  ,   9.55701 ,   1.70996 ],
       [-18.8699  ,   9.43143 ,   1.70856 ],
       [-18.8518  ,   9.27494 ,   1.70106 ],

In [21]:
np.genfromtxt?